# Notebook 1 : Transformer

Le but de ce notebook est d'importer les données des datasets iSarcasmEval et de tester les transformer (tel que donné par *“Attention Is All You Need”*) afin de mesurer l'accuracy de base du projet et quantifier les futures améliorations

### Etape 1 : Importer les librairies utiles au projet

In [1]:
import torch as torch
import pandas as pd

Test de Pytorch

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


### Etape 2 : Chargeons les données

In [13]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset


class iSarcasmDataset(Dataset):
  def __init__(self, sentence, labels):
        self.sentence = sentence
        self.labels = labels

  def __len__(self):
        return len(self.sentence)

  def __getitem__(self, index):
        x = self.sentence[index]
        y = self.labels[index]

        return x, y
    
    

print("--- Chargement du dataset ---")


#On donne les informations du dataset
directory = "dataset/"
train_name = "train.En.csv"
test_name = "task_A_En_test.csv"


#On défini les paramètres du dataloader
params = {'batch_size': 100,
          'shuffle': False,
          'num_workers': 1}


#On créé nos données de test
panda_Train = pd.read_csv(directory+train_name, index_col=[0])
X_train = panda_Train['tweet']
y_train = panda_Train['sarcastic']


#On créé notre dataset pyTorch
train_iSarcasm = iSarcasmDataset(X_train, y_train)
training_generator = DataLoader(train_iSarcasm, **params)


#On regarde que tout soit bien chargé
for i, (batch, labels) in enumerate(training_generator):
    if i > 0:
        break
    print("Batch size : "+str(len(batch)))
    
print("--- Chargement du dataset terminé ---")

--- Chargement du dataset ---
Batch size : 100
--- Chargement du dataset terminé ---


On va maintenant entrainer un transformer de base sur nos données :